In [2]:
from reservoir_rls_multires import *
import matplotlib.pyplot as plt
from lorenz63 import *
from scipy.signal import welch, periodogram
from sklearn.preprocessing import StandardScaler
import cma

In [3]:
get_data = True
data_length = 1000000
step = 0.05
f_s = 1/step
scale = 0.01
slow_var = 48/28
r_t = lambda x: r_t_cosine(x)
dx_dt = lambda x, time, r_t: dxdt_lorenz(x, time, r_t)
if get_data:
    lorenz_data_cosine = getLorenzData(data_length, r_t, dx_dt, sample_tau = step)
    np.savetxt('lorenz_data_cosine_step%0.2f.csv' %(step), lorenz_data_cosine, delimiter = ',')
else:
    lorenz_data_cosine = np.loadtxt('lorenz_data_cosine_step%0.2f.csv' %(step), delimiter = ',')
times = np.arange(lorenz_data_cosine.shape[0])*step
external_data = r_t(times)
"""
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(111,projection = '3d')
ax.plot3D(lorenz_data_rossler[:,0],lorenz_data_rossler[:,1],lorenz_data_rossler[:,2])
plt.show()
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(111,projection = '3d')
ax.plot3D(lorenz_data_rossler[:,3],lorenz_data_rossler[:,4],lorenz_data_rossler[:,5])
plt.show()
"""
scaled_data = lorenz_data_cosine
scaled_data = np.ascontiguousarray(scaled_data)
"""
scaled_data_external = SS.fit_transform(lorenz_data_rossler[:,4].reshape(-1,1))
data_external = lorenz_data_rossler[:,4]

Wn_xy = 0.0075
Wn_z = Wn_xy
lorenz_data_split = separate_lorenz_2scale(lorenz_data_rossler[:,:3], f_s, Wn_xy, Wn_z, filter_order = 10)
data_external = lorenz_data_split[init_delay:,5]
lorenz_data_rossler  = lorenz_data_rossler[init_delay:]
scaled_data_external = SS.fit_transform(data_external.reshape(-1,1))
plt.plot(lorenz_data_rossler[:,4])
plt.plot(data_external-np.mean(data_external))
plt.show()
"""

'\nscaled_data_external = SS.fit_transform(lorenz_data_rossler[:,4].reshape(-1,1))\ndata_external = lorenz_data_rossler[:,4]\n\nWn_xy = 0.0075\nWn_z = Wn_xy\nlorenz_data_split = separate_lorenz_2scale(lorenz_data_rossler[:,:3], f_s, Wn_xy, Wn_z, filter_order = 10)\ndata_external = lorenz_data_split[init_delay:,5]\nlorenz_data_rossler  = lorenz_data_rossler[init_delay:]\nscaled_data_external = SS.fit_transform(data_external.reshape(-1,1))\nplt.plot(lorenz_data_rossler[:,4])\nplt.plot(data_external-np.mean(data_external))\nplt.show()\n'

In [4]:
def phase_min_func(delay, truth, filtered):
    delay = round(delay)
    truth_delayed = truth[:truth.shape[0]-delay]
    filtered_delayed = filtered[delay:]
    val = -np.mean(truth_delayed * filtered_delayed)
    return val
def min_func_wphase(x, mask, base_data, f_s, true_external_data,\
    base_res, num_tests, num_nodes, pred_length, train_length, scale = True, 
    external_output = True):
    init_delay = 20000
    max_freq = 0.15
    min_freq = 0.001
    Wn_xy = x[0]/10*(max_freq-min_freq)+min_freq
    Wn_z = Wn_xy
    data_split = separate_lorenz_2scale(base_data, f_s, Wn_xy, Wn_z, filter_order = 10)
    base_external_data = data_split[init_delay:,-1]
    num_delays = 10000
    z_centered = base_external_data - np.mean(base_external_data)
    min_func   = lambda delay: phase_min_func(delay, true_external_data[init_delay:], z_centered)
    func_vals = np.zeros(num_delays)
    for i in range(num_delays):
        func_vals[i] = min_func(i)
    min_delay = np.argmin(func_vals)
    data = base_data[init_delay:base_data.shape[0]-min_delay]
    external_data = base_external_data[min_delay:]
    if scale:
        SS = StandardScaler()
        external_data = SS.fit_transform(external_data.reshape(-1,1))
    funval = vt_min_function_norm_external(data,external_data, x[1:], mask, base_res.Win, base_res.A, \
        num_tests = num_tests,  num_nodes = num_nodes, pred_length = pred_length, train_length = train_length,\
        external_output = external_output)
    return funval
def min_func_wtruth(x, mask, base_data, f_s, true_external_data,\
    base_res, num_tests, num_nodes, pred_length, train_length, scale = True, 
    external_output = True):
    init_delay = 0
    data = base_data[init_delay:]
    external_data = true_external_data[init_delay:]
    if scale:
        SS = StandardScaler()
        external_data = SS.fit_transform(external_data.reshape(-1,1))
    funval = vt_min_function_norm_external(data,external_data, x, mask, base_res.Win, base_res.A, \
        num_tests = num_tests,  num_nodes = num_nodes, pred_length = pred_length, train_length = train_length,\
        external_output = external_output)
    return funval
num_nodes = 360
num_tests = 200
train_length = 3000
sync_length = 500
pred_length = 500
res_seed = 1
base_res = reservoir(4,num_nodes,input_weight = 1, spectral_radius = 1, seed = res_seed) #Generate a reservoir
mask = ['input_weight', 'regularization', 'leakage', 'forget']
x0 = np.array([6,4,0,9])
min_func_base = lambda x: min_func_wtruth(x, mask, scaled_data, f_s, external_data,\
    base_res, num_tests, num_nodes, pred_length, train_length)
sigma = 2

In [5]:
opts = cma.CMAOptions()
opts.set('popsize',10*x0.size) # Set number of samples per generation
"""
Set bounds on parameters. IMPORTANT: The mean returned by cma-es is
the mean BEFORE the boundary function is applied, so the mean may not
lie in the domain set by bounds. To obtain the true sample mean requires 
downloading the cma-es package from github and editing one of the 
functions. Ask me if you need to do this.
"""
opts.set('bounds', [0,10]) 
opts.set('seed', 5) # Seed for the initial samples
"""
File where results are saved. IMPORTANT: Full covariance matrix is 
NOT saved, nor are the exact samples. If these need to be saved, one
will also have to download from github and make some edits. Again,
ask me.
"""
opts.set('verb_filenameprefix','cmaes_lorenz_cosine_wtruthout_scaled_res%d\\' % res_seed)
results = cma.fmin(min_func_base, x0, sigma, options = opts) # Run the algorithm

(20_w,40)-aCMA-ES (mu_w=11.3,w_1=16%) in dimension 4 (seed=5, Tue Jan 26 10:39:46 2021)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     40 -5.900000000000000e+01 1.0e+00 1.95e+00  2e+00  2e+00 15:49.4
    2     80 -6.000000000000000e+01 1.4e+00 1.81e+00  1e+00  2e+00 34:42.4
    3    120 -6.100000000000000e+01 2.1e+00 1.55e+00  7e-01  1e+00 54:04.7
    4    160 -5.850000000000000e+01 2.6e+00 1.30e+00  4e-01  1e+00 79:20.1


KeyboardInterrupt: 

In [ ]:
data_length = 100000
step = 0.05
f_s = 1/step
scale = 0.01
slow_var = 48/28
r_t = lambda x: r_t_const(x)
dx_dt = lambda x, time, r_t: dxdt_lorenz(x, time, r_t)
lorenz_data = getLorenzData(data_length, r_t, dx_dt, sample_tau = step)
scaled_data = np.ascontiguousarray(lorenz_data)
plt.plot(lorenz_data[:1000,0])
plt.show()

In [14]:
from scipy.special import expit
res_rossler = reservoir(4, 360, input_weight = 0.01, leakage =  0, \
                        regularization = 1e-7, forget = 1, seed = 1)
train_length = 3000
sync_length = 500
pred_length = 500
num_tests = 200
# results = cross_validation_performance_resync_wextern(scaled_data, scaled_data_external, res_rossler, num_tests, sync_length, train_length, pred_length, \
#          seed = 10, errormax = 3.2, train_method = 'Normal', progress = True, plot = False)
results = cross_validation_performance_resync_wextern(new_scaled_data, new_scaled_data_external, \
         res_rossler, num_tests, sync_length, train_length, pred_length, \
         seed = 10, errormax = 3.2, train_method = 'Normal', progress = True, external_output = False)
print(results)
print(np.median(results))
results = cross_validation_performance_resync_wextern(new_scaled_data, new_scaled_data_external, \
         res_rossler, num_tests, sync_length, train_length, pred_length, \
         seed = 10, errormax = 3.2, train_method = 'Normal', progress = True, external_output = True)
print(results)
print(np.median(results))
results = cross_validation_performance_resync_wextern(new_scaled_data, new_data_external, res_rossler, num_tests, sync_length, train_length, pred_length, \
         seed = 10, errormax = 3.2, train_method = 'Normal', progress = True, external_output = False)
print(results)
print(np.median(results))
results = cross_validation_performance_resync_wextern(new_scaled_data, new_data_external, res_rossler, num_tests, sync_length, train_length, pred_length, \
         seed = 10, errormax = 3.2, train_method = 'Normal', progress = True, external_output = True)
print(results)
print(np.median(results))

  0%|▍                                                                                 | 1/200 [00:00<00:22,  8.77it/s]

[ 21.  44.  29.  37.  32.  11.   2.  67.   5.  25.  61.  40.   9.   5.
  22.   8.  32.  47.   7.  21.  19. 158.  40.  41.  29.  51.  25.   5.
  77.   7.  36.  39.  57.   6.  18.  64.   7.  34.  33.  56.  23.  31.
  64.  44.  68.  14.  47.  43.  24.  13.  13.  25.  54.  74.  23.  53.
   8.   6.  18.   6.  25.  16.   3.  14.  10.  10.   5.  15.  22.  24.
  10.  20.   9.   6.  42.  29.   9.  38.   4.  17.  33.  32.   8.  16.
  13.  24.  54.   8.   8.  30.  63.  18.  22.  10.  34.  33.   9.   6.
   1.   3.   2.  16.   4.   2.  87.  15.  22.  50.  18.  26.  19.  31.
  18.   5.   7.  35.  82. 108.  15.   7.  42.   7.  51.  39.  63.   9.
  12.  23.  13.  27.   6.  42.  45.  13.  37.   7.  13. 117.  15.  35.
  78.  20.  19.  88.  97.  14.   5.  11.   8.  25.  58.  13.  20.  37.
   8.  46.   3.   6.  29.  17.  21.  11.  21.  24.  49.  33.  17.   5.
  18.  17.  19.  10.   5.  69.  38.   5.  17.  15.  62.  72.   6.  10.
   9.  36.  22.  75.  74.  20. 143.  74.  48.  61.   7.  95.  33.   6.
  26. 

  0%|▍                                                                                 | 1/200 [00:00<00:21,  9.34it/s]

[ 21.  12.  29.  35.  21.  11.   2.  67.   5.  24.  63.  39.   9.   5.
  22.   8.  33.  47.   7.  20.  17. 114.  41.  41.  29.  50.  26.   5.
  76.   6.  56.  50.  58.   6.  18.  75.   7.  33.  33.  58.  23.  31.
  53.  44.  41.  15.  46.  44.  33.  11.  12.  25.  54.  74.  22.  38.
   8.   6.  18.   6.  24.  16.   3.  14.   9.  10.   5.  40.  22.  25.
  10.  21.   8.   6.  40.  29.   9.  28.   4.  17.  32.  32.   8.  16.
  13.  24.  29.   8.   8.  29.  63.  18.  21.  11.  21.  33.   9.   6.
   1.   3.   2.  15.   4.   2.  65.  14.  22.  50.  19.  28.  19.  18.
  18.   5.   7.  38.  81. 109.  17.   7.  44.   7.  50.  39.  64.   9.
  13.  23.  11.  27.   6.  41.  45.  13.  36.   7.  13. 118.  14.  36.
  78.  20.  22.  88.  53.  14.   5.  11.   8.  25.  57.  13.  20.  22.
   7.  45.   3.   6.  29.  17.  21.  11.  22.  49.  49.  33.  25.   5.
  18.  17.  10.   9.   5.  69.  38.   5.  17.  26.  62.  83.   6.   9.
   9.  36.  22.  76.  45.  20. 146.  73.  44.  61.   7.  53.  30.   6.
  27. 

  0%|▍                                                                                 | 1/200 [00:00<00:24,  8.19it/s]

[ 24.  10.  27.  22.  34.   7.   3.  65.  17.   5. 127.  25.  18.   5.
  69.   7.  33.  60.  20.  20.  38.  70.  38.  40.  65.  49.  26.   7.
  26.   6.  34.  53.  57.   6.  33.  17.   9.  10.  30.  44.  50. 112.
  53.  43.  41.  11.   7.  71.  12.  11.  24.  66.  57.  64.  36.  56.
  10.   6.   7.  33.  22.  16.   3.  14.   6.  10.   9.  12.  23.  22.
   9.  22.  22.   5.  27.  15.  13.  66.   4.  21.  61.  10.  20.  18.
  36.  10.  16.   3.  38.  47.  64.  18.   9.  24.  21.  35.   9.  21.
   5.   4.   5.  16.   4.   3.  28.  14.  44.  52.  32.  29.  51.  30.
  23.  13.   8.  51. 108. 122.  17.   5.  48.   7.  67.  40.  22.  10.
  12.  23.  22.  38.   6.  41.  17.  65.  38.   8.  15.  79.  14.  31.
 103.  21.  18. 101.  52.  12.   6.  15.   8.  26. 112.  27.  19.  10.
   4.  22.   3.   5.  15.  17.   8.  32.  60.  23.  92.  32.  15.   4.
  10.  31.  10.  21.   8.  23.  40.   7.  15.  21.  62.  38.   6.  12.
   9.  36.  13.  39.  58.  18.  74.  72.  98.  58.  19.  53.  18.   7.
  38. 

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:22<00:00,  8.91it/s]

[ 23.  10.  27.  21.  35.   8.   6.  67.  17.   4. 103.  25.  19.   5.
  24.   8.  75.  92.  20.  22.  18.  70.  38.  32.  65.  49.  25.   7.
  26.   6.  34.  54.  57.   5.  33.  17.   3.  10.  30.  44.  54.  33.
  52.  43.  41.  11.   6.  70.  12.  10.  35.  66.  57.  65.  36.  92.
  10.   5.   7.  44.  23.  16.   2.  14.   7.  10.   6.  13.  23.  22.
   9.  23.  23.   5.  27.  15.  14.  69.   3.  21.  34.  21.  29.  15.
  37.  10.  15.   3.  51.  48.  64.  18.  20.  24.  21.  35.   9.  21.
   2.   4.   1.  17.   4.   6.  30.  13.  44.  52.  44.  29.  42.  30.
  20.  13.   7.  92. 109. 122.  17.   6.  46.   7. 122.  42.  22.  10.
  15.  23.  16.  37.   7.  41.  17.  64.  38.   7.   4.  79.  16.  31.
 103.  22.  20. 100.  51.  11.   8.  14.   7.  25. 112.   6.  19.  22.
   4.  22.   2.   5.  16.  17.   8.  34.  60.  23. 102.  32.  14.   3.
  10.  31.  11.  33.   9.  57.  40.   5.  15.  22.  62.  37.   6.  11.
  18.  36.  13.  39.  59.  18. 116.  72.  46.  58.  13.  53.  19.   7.
  38. 

In [4]:
res_rossler = reservoir(3, 360, input_weight = 0.01, leakage =  0, \
                        regularization = 1e-7, forget = 1, seed = 1)
train_length = 3000
sync_length = 500
pred_length = 500
num_tests = 200
# results = cross_validation_performance_resync_wextern(scaled_data, scaled_data_external, res_rossler, num_tests, sync_length, train_length, pred_length, \
#          seed = 10, errormax = 3.2, train_method = 'Normal', progress = True, plot = False)
results = cross_validation_performance_resync(scaled_data, res_rossler, num_tests, sync_length, train_length, pred_length, \
         seed = 10, errormax = 3.2, train_method = 'Normal', progress = True)
print(results)
print(np.median(results))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:21<00:00,  9.27it/s]

[ 20.   8.  10.   7.  16.   9.  10.  25.  18.  24.   8.   2.   9.   8.
 122.  12.  19.   6.  10.   3.  24.   9.  17.  28.  17.   5.   4.   5.
  13.   9.  11.   8.  15.  19.  13.  21.  12.  11.  17.  39.  19.  13.
  10.   4.   8.  20.   6.  26.  21.   9.  16.   9.   3.  16.   2.  16.
  10.  10.  37.   5.   5.  14.  12.  15.   7.   7.   6.   5.  12.  12.
  11.   9.  24.  22.  17.   5.  47.   4.  12.  14.  15.  89.  27.  27.
  14.   9.   9.  18.  17.   6.   4.   8.   6.   4.   4.   9.  10.   5.
  14.   9.  69.   4.   8.  13.   8.   8.  17.   2.   5.   9.   5.   8.
   7.  39.   5.   5.  31.  20.  15.   3.  15.  19.   4.  19.   9.   6.
   6.  13.  12.  15.  19.   8.   5.  22.   9.   8.   7.   3.  45.  11.
   5.   6.  27.  19.  23.  12.   8.  21.   8.   4.  16.  15.   8.   8.
   7.  27.   8.   7.   3.  10.   5.  18.   8.  13.   4.  10.  74.  13.
  14.  12.   1.  10.  24.   9.  20.  30.   6.  25.  46.  15.   5.  15.
  18.  19.  21.  10.   6.   7.  17.  16.   5.   8.   9.  11.  25.   7.
  22. 

In [ ]:
np.array([[1,2,3,4],[5,6,7,8]]).T/np.array([1,2])

In [ ]:
get_data = True
data_length = 2500000
step = 0.05
f_s = 1/step
scale = 1
r_t = lambda x: r_t_const(x)
dx_dt = lambda x, time, r_t: dxdt_rossler(x, time, r_t, scale = scale)
if get_data:
    rossler_data = getLorenzData(data_length, r_t, dx_dt, sample_tau = step)
    np.savetxt('rossler_data_step')

In [ ]:
num_nodes = 100
num_tests = 100
train_length = 10000
sync_length = 500
pred_length = 2000
res_seed = 1
base_res = reservoir(1,num_nodes,input_weight = 1, spectral_radius = 1, seed = res_seed) #Generate a reservoir
mask = ['input_weight', 'regularization', 'leakage']
x0 = np.array([6,6,6])
# Set the minimization function. This function takes num_tests training
# and validation data sets and trains the reservoir to predict each validation set
# after being trained on the corresponding training set. The negative median valid
# time is returned to be minimized.
min_func = lambda x: vt_min_function_norm(rossler_data[:,1], x, mask, base_res.Win, base_res.A, \
     num_tests = num_tests,  num_nodes = num_nodes, pred_length = pred_length, train_length = train_length)
sigma = 2

In [ ]:
opts = cma.CMAOptions()
opts.set('popsize',10*x0.size) # Set number of samples per generation
"""
Set bounds on parameters. IMPORTANT: The mean returned by cma-es is
the mean BEFORE the boundary function is applied, so the mean may not
lie in the domain set by bounds. To obtain the true sample mean requires 
downloading the cma-es package from github and editing one of the 
functions. Ask me if you need to do this.
"""
opts.set('bounds', [0,10]) 
opts.set('seed', 5) # Seed for the initial samples
"""
File where results are saved. IMPORTANT: Full covariance matrix is 
NOT saved, nor are the exact samples. If these need to be saved, one
will also have to download from github and make some edits. Again,
ask me.
"""
opts.set('verb_filenameprefix','cmaes_rossler_y_norm_wleakage_res%d\\' % res_seed)
results = cma.fmin(min_func, x0, sigma, options = opts) # Run the algorithm

In [ ]:
from scipy.special import expit
expit(10)

In [ ]:
res_base = reservoir(4, 360, input_weight = 0.017, regularization = 1e-10, forget = 1)
train_length = 3000
sync_length = 300
pred_length = 500
num_tests = 100
results = cross_validation_performance_resync(rossler_data_w_lowfreq, res_base, num_tests, sync_length, train_length, pred_length, \
         seed = 10, errormax = 3.2, train_method = 'Normal', progress = True)
print(results)
print(np.mean(results))

In [ ]:
np.concatenate((np.array([1,2]).reshape(-1,1),np.array([])), axis = 1)

In [ ]:
data_length = 100000
step = 0.05
r_t = lambda x: r_t_const(x)
dx_dt = lambda x, time, r_t: dxdt_lorenz(x, time, r_t)
lorenz_data_base = getLorenzData(data_length, r_t, dx_dt, sample_tau = step)

In [ ]:
print(np.mean(lorenz_data_base, axis = 0))
print(np.std(lorenz_data_base, axis = 0))

In [ ]:
res_base = reservoir(3, 300, input_weight = -0.06, input_bias = 0.04, regularization = 1e-7, forget = 0.999)
train_length = 1000
sync_length = 500
pred_length = 500
num_tests = 100
results = cross_validation_performance_resync(lorenz_data_rossler[:,:3], res_base, num_tests, sync_length, train_length, pred_length, \
         seed = 10, errormax = 3.2, train_method = 'Normal', progress = True, plot = False)
print(results)
print(np.mean(results))

In [ ]:
d_res = double_reservoir(6, 400, input_weight = [0.017,0.017], regularization = [1e-10,1e-10], forget = [1,0.99])
train_length = 1000
sync_length = 500
pred_length = 500
num_tests = 50
data = np.append(lorenz_data_rossler[:,:3], lorenz_data_rossler[:,:3], axis = 1)
target_data = lorenz_data_split
results = cross_validation_performance_resync_decompose(data, target_data, d_res, num_tests, sync_length, train_length, pred_length, \
         seed = 5, errormax = 3.2, train_method = 'Normal', progress = True, plot = False)
print(results)
print(np.mean(results))